In [1]:
import cv2, time, threading

caps = {
    "cam0": cv2.VideoCapture(3,cv2.CAP_V4L2),
    "cam1": cv2.VideoCapture(5,cv2.CAP_V4L2)
}

for c in caps.values():
    c.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
    c.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    c.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    c.set(cv2.CAP_PROP_FPS, 60)

last = time.time()
cnt = {"cam0":0, "cam1":0}
i = 0
while i<10:
    for name, cap in caps.items():
        ret, frame = cap.read()
        #cv2.imshow("testing",frame)
        if ret:
            cnt[name]+=1

    now = time.time()
    if now - last >= 1:
        print("FPS:", cnt)
        cnt = {"cam0":0, "cam1":0}
        i+=1
        last = now
for c in caps.values():
    c.release()
#cv2.destroyAllWindows()

KeyboardInterrupt: 

In [3]:
import cv2, numpy as np, time, os, sys, threading, subprocess

# =============== CONFIG ===============
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11

CAM_SOURCES = {
    "Mobile":"http://192.168.137.110:8080/video",
    "Kreo1": 0,
    "Kreo2": 4
}

def create_detector():
    """Setup AprilTag detector with tuned parameters."""
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 5
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 4.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

class CameraWorker(threading.Thread):
    def __init__(self, name, src, detector):
        super().__init__(daemon=True)
        self.name = name
        self.src = src
        self.cap = cv2.VideoCapture(src, cv2.CAP_V4L2)

        try:
            self.cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
            self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        except Exception:
            pass
        if isinstance(src, int):
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            self.cap.set(cv2.CAP_PROP_FPS, 60)
        self.detector = detector
        self.lock = threading.Lock()
        self.latest_frame = None
        self.latest_ts = 0.0
        self.running = True
        self.opened = self.cap.isOpened()
        if not self.opened:
            print(f"[{self.name}] Error cannot open source {src}")

    def run(self):
        while self.running and self.opened:
            ret, frame = self.cap.read()
            if not ret:
                time.sleep(0.01)
                continue
            ts = time.time()
            with self.lock:
                self.latest_frame = frame
                self.latest_ts = ts
            time.sleep(0.001)

    def read_latest(self):
        with self.lock:
            if self.latest_frame is None:
                return None, 0.0
            return self.latest_frame.copy(), self.latest_ts
    
    def stop(self):
        self.running = False
        try: self.cap.release()
        except: pass


def get_camera_selection():
    print("\n=== Multi-Camera Live View Setup ===")
    print("Select cameras to open (comma separated):")
    print("1. Kreo Webcam #1")
    print("2. Kreo Webcam #2")
    print("3. Mobile IP Webcam")
    print("Example: 1,2 or 1,3 or 1,2,3")
    user_in = input("Cameras to open: ").strip()
    choices = [x.strip() for x in user_in.split(",") if x.strip()]
    selected = []
    for c in choices:
        if c == "1":
            selected.append(("Kreo1", CAM_SOURCES["Kreo1"]))
        elif c == "2":
            selected.append(("Kreo2", CAM_SOURCES["Kreo2"]))
        elif c == "3":
            selected.append(("Mobile", CAM_SOURCES["Mobile"]))
        else:
            print(f"[WARN] Ignoring invalid entry: {c}")
    if not selected:
        print("[ERROR] No valid cameras selected. Exiting.")
        sys.exit(1)
    return selected

if __name__ == "__main__":
    selected = get_camera_selection()

    workers = {}
    for name, src in selected:
        w = CameraWorker(name, src, create_detector())
        w.start()
        workers[name] = w
        time.sleep(0.05)
    
    # After tuning show live preview with settings applied
    print("[INFO] Preview without fine tuning. Press ESC to exit preview windows.")
    last_ts = {name: 0.0 for name in workers}
    fps_counter = {name: 0 for name in workers}
    fps = {name: 0.0 for name in workers}
    last_fps_update = time.time()

    try:
        while True:
            now = time.time()
            timestamps = {}

            # --- Collect frames from all cameras ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                timestamps[name] = ts

                # --- FPS update ---
                if ts != last_ts[name]:
                    fps_counter[name] += 1
                    last_ts[name] = ts

                if now - last_fps_update >= 1.0:
                    fps[name] = fps_counter[name]
                    fps_counter[name] = 0

            if now - last_fps_update >= 1.0:
                last_fps_update = now

            # --- Drift calculation ---
            if len(timestamps) > 1:
                tvals = np.array(list(timestamps.values()))
                drift_ms = (tvals.max() - tvals.min()) * 1000.0
            else:
                drift_ms = 0.0

            # --- Draw every camera independently ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                # AprilTag overlay
                corners, ids, _ = w.detector.detectMarkers(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
                if ids is not None and len(ids) > 0:
                    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

                # resolution text
                h, wid = frame.shape[:2]
                cv2.putText(frame, f"{name} {wid}x{h}", (10, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"FPS: {fps[name]:.0f}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"Drift: {drift_ms:.1f} ms", (10, 75),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

                cv2.imshow(f"Tuned - {name}", frame)

            # Exit on ESC
            if cv2.waitKey(1) & 0xFF == 27:
                break

            time.sleep(0.001)

    except KeyboardInterrupt:
        pass

    finally:
        for w in workers.values():
            w.stop()
        cv2.destroyAllWindows()




=== Multi-Camera Live View Setup ===
Select cameras to open (comma separated):
1. Kreo Webcam #1
2. Kreo Webcam #2
3. Mobile IP Webcam
Example: 1,2 or 1,3 or 1,2,3
[INFO] Preview without fine tuning. Press ESC to exit preview windows.


In [7]:
import cv2, time, threading, subprocess, json, os, numpy as np, sys

# =============== CONFIG ===============
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11

CAM_SOURCES = {
    "mobile":"http://192.168.137.110:8080/video",
    "kreo1": 0,
    "kreo2": 4
}

# parameter grids. Tweak if needed
EXPOSURES = [80, 120, 160, 200, 240, 300]
FOCUSES = [80, 120, 160, 200, 240, 280, 320, 360]
GAINS = [0, 6, 12, 18]
BRIGHTNESSES = [0, 8, 16, 24]

# evaluation parameters
EVAL_SECONDS = 1
SAMPLE_SLEEP = 0.02

OUTPUT_DIR = "./camera_tune_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# AprilTag detector setup
def create_detector():
    """Setup AprilTag detector with tuned parameters."""
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 7
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 6.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

class CameraWorker(threading.Thread):
    def __init__(self, name, src, detector):
        super().__init__(daemon=True)
        self.name = name
        self.src = src
        self.cap = cv2.VideoCapture(src,cv2.CAP_V4L2)

        try:
            self.cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
            self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        except Exception:
            pass
        if isinstance(src, int):
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            self.cap.set(cv2.CAP_PROP_FPS, 60)
        self.detector = detector
        self.lock = threading.Lock()
        self.latest_frame = None
        self.latest_ts = 0.0
        self.running = True
        self.opened = self.cap.isOpened()
        if not self.opened:
            print(f"[{self.name}] Error cannot open source {src}")

    def run(self):
        while self.running and self.opened:
            ret, frame = self.cap.read()
            if not ret:
                time.sleep(0.01)
                continue
            ts = time.time()
            with self.lock:
                self.latest_frame = frame
                self.latest_ts = ts
            time.sleep(0.001)

    def read_latest(self):
        with self.lock:
            if self.latest_frame is None:
                return None, 0.0
            return self.latest_frame.copy(), self.latest_ts
    
    def stop(self):
        self.running = False
        try: self.cap.release()
        except: pass

# v4l2 control helper
def v4l2_set(dev_idx, control, value):
    dev = f"/dev/video{dev_idx}"
    cmd = ["v4l2-ctl", "-d", dev, "-c", f"{control}={value}"]
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except subprocess.CalledProcessError:
        return False

def v4l2_get(dev_idx, control):
    dev = f"/dev/video{dev_idx}"
    cmd = ["v4l2-ctl", "-d", dev, "--get-ctrl", control]
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.DEVNULL).decode().strip()
        return out
    except subprocess.CalledProcessError:
        return None
    
def evaluate_setting(worker, sample_duration=EVAL_SECONDS):
    start = time.time()
    end = start + sample_duration
    tag_counts = []
    frames = 0
    t0 = time.time()
    while time.time()<end:
        frame, ts = worker.read_latest()
        if frame is None:
            time.sleep(SAMPLE_SLEEP)
            continue
        frames += 1
        corners, ids, _ = worker.detector.detectMarkers(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        n = 0 if ids is None else len(ids)
        tag_counts.append(n)
        time.sleep(SAMPLE_SLEEP)
    elapsed = time.time()- t0
    avg_tags = float(np.mean(tag_counts)) if tag_counts else 0.0
    std_tags = float(np.std(tag_counts)) if tag_counts else 0.0
    fps = frames / elapsed if elapsed>0 else 0.0
    return { "avg_tags": avg_tags, "std_tags": std_tags, "fps": fps, "samples": len(tag_counts)}

def tune_camera(dev_idx, worker, brief_name):
    if not isinstance(worker.src, int):
        print(f"[{brief_name}] Skipping tuning for non-local source {worker.src}")
        return None

    print(f"[{brief_name}] Starting adaptive tuning (~5 minutes)...")

    # ---------- Disable auto controls ----------
    try:
        v4l2_set(dev_idx, "focus_automatic_continuous", 0)
        v4l2_set(dev_idx, "auto_exposure", 1)
        v4l2_set(dev_idx, "exposure_dynamic_framerate", 0)
    except Exception:
        pass

    # ---------- Utility ----------
    def neighbors(val, lst, radius=1):
        try:
            i = lst.index(val)
        except ValueError:
            return [val]
        lo = max(0, i-radius)
        hi = min(len(lst)-1, i+radius)
        return lst[lo:hi+1]

    def set_all(exp, foc, g, b):
        v4l2_set(dev_idx, "exposure_time_absolute", int(exp))
        time.sleep(0.04)
        v4l2_set(dev_idx, "focus_absolute", int(foc))
        time.sleep(0.03)
        v4l2_set(dev_idx, "gain", int(g))
        time.sleep(0.02)
        v4l2_set(dev_idx, "brightness", int(b))
        time.sleep(0.05)

    # ---------- Stage 1: Coarse grid ----------
    def pick_coarse(lst, n=3):
        if len(lst) <= n:
            return lst
        idxs = [int(round(i*(len(lst)-1)/(n-1))) for i in range(n)]
        return [lst[i] for i in idxs]

    exp_c = pick_coarse(EXPOSURES, 3)
    foc_c = pick_coarse(FOCUSES, 3)
    gain_c = pick_coarse(GAINS, 3)
    bright_c = pick_coarse(BRIGHTNESSES, 3)

    coarse_combos = [(e,f,g,b) for e in exp_c for f in foc_c for g in gain_c for b in bright_c]

    print(f"[{brief_name}] Coarse grid: {len(coarse_combos)} combos")

    coarse_results = []
    for idx, (e, f, g, b) in enumerate(coarse_combos):
        print(f"[{brief_name}] [Coarse {idx+1}/{len(coarse_combos)}] e={e}, f={f}, g={g}, b={b}     ", end="\r")
        set_all(e, f, g, b)
        stats = evaluate_setting(worker)

        score = stats["avg_tags"] * 100.0 + stats["fps"] * 0.2 - stats["std_tags"] * 10.0
        coarse_results.append((score, (e, f, g, b), stats))

    print("")  # newline

    coarse_results.sort(reverse=True, key=lambda x: x[0])
    TOP_K = 4
    topk = coarse_results[:TOP_K]

    print(f"[{brief_name}] Top coarse candidates:")
    for score, combo, stats in topk:
        print("   ", combo, f"score={score:.1f}", stats)

    # ---------- Stage 2: refinement ----------
    refined_results = []
    for score0, (exp0, foc0, g0, b0), stats0 in topk:
        exp_n = neighbors(exp0, EXPOSURES, radius=1)
        foc_n = neighbors(foc0, FOCUSES, radius=1)
        g_n   = neighbors(g0,  GAINS, radius=1)
        b_n   = neighbors(b0,  BRIGHTNESSES, radius=1)

        local = [(e,f,g,b) for e in exp_n for f in foc_n for g in g_n for b in b_n]

        print(f"[{brief_name}] Refining around {exp0, foc0, g0, b0} → {len(local)} combos")
        for idx, combo in enumerate(local):
            e,f,g,b = combo
            print(f"[{brief_name}] [Refine {idx+1}/{len(local)}] e={e}, f={f}, g={g}, b={b}     ", end="\r")
            set_all(e,f,g,b)
            stats = evaluate_setting(worker)
            score = stats["avg_tags"] * 100.0 + stats["fps"] * 0.2 - stats["std_tags"] * 10.0
            refined_results.append((score, combo, stats))
        print("")

    # ---------- Decide best ----------
    all_results = coarse_results + refined_results
    all_results.sort(reverse=True, key=lambda x: x[0])

    best_score, best_combo, best_stats = all_results[0]
    e,f,g,b = best_combo

    print("")
    print(f"[{brief_name}] BEST SETTINGS:")
    print(f"   exposure={e}, focus={f}, gain={g}, brightness={b}")
    print(f"   score={best_score:.1f}, stats={best_stats}")

    # ---------- Apply final ----------
    set_all(e,f,g,b)

    # ---------- Save ----------
    out = {
        "exp": e, "focus": f, "gain": g, "brightness": b,
        "score": best_score,
        "stats": best_stats
    }
    out_file = os.path.join(OUTPUT_DIR, f"best_camera_settings_{brief_name}.json")
    with open(out_file, "w") as f:
        json.dump(out, f, indent=2)
    print(f"[{brief_name}] Saved → {out_file}")

    return out

def get_camera_selection():
    print("\n=== Multi-Camera Live View Setup ===")
    print("Select cameras to open (comma separated):")
    print("1. Kreo Webcam #1")
    print("2. Kreo Webcam #2")
    print("3. Mobile IP Webcam")
    print("Example: 1,2 or 1,3 or 1,2,3")
    user_in = input("Cameras to open: ").strip()
    choices = [x.strip() for x in user_in.split(",") if x.strip()]
    selected = []
    for c in choices:
        if c == "1":
            selected.append(("kreo1", CAM_SOURCES["kreo1"]))
        elif c == "2":
            selected.append(("kreo2", CAM_SOURCES["kreo2"]))
        elif c == "3":
            selected.append(("mobile", CAM_SOURCES["mobile"]))
        else:
            print(f"[WARN] Ignoring invalid entry: {c}")
    if not selected:
        print("[ERROR] No valid cameras selected. Exiting.")
        sys.exit(1)
    return selected

# ==================== MAIN ====================
if __name__ == "__main__":
    selected = get_camera_selection()

    workers = {}
    for name, src in selected:
        w = CameraWorker(name, src, create_detector())
        w.start()
        workers[name] = w
        time.sleep(0.05)

    # small warmup
    print("[INFO] Warmup for 1.2 seconds to let cameras settle...")
    time.sleep(1.2)
    
    for name, w in list(workers.items()):
        if isinstance(w.src,int):
            try:
                dev_idx = int(w.src)
            except Exception:
                print(f"[{name}] Invalid device index for tuning: {w.src}")
                continue
            best = tune_camera(dev_idx, w, name)
        else:
            print(f"[{name}] Not a local device; skipping tuning.")

    # After tuning show live preview with settings applied
    print("[INFO] Tuning complete. Press ESC to exit preview windows.")
    last_ts = {name: 0.0 for name in workers}
    fps_counter = {name: 0 for name in workers}
    fps = {name: 0.0 for name in workers}
    last_fps_update = time.time()

    try:
        while True:
            now = time.time()
            timestamps = {}

            # --- Collect frames from all cameras ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                timestamps[name] = ts

                # --- FPS update ---
                if ts != last_ts[name]:
                    fps_counter[name] += 1
                    last_ts[name] = ts

                if now - last_fps_update >= 1.0:
                    fps[name] = fps_counter[name]
                    fps_counter[name] = 0

            if now - last_fps_update >= 1.0:
                last_fps_update = now

            # --- Drift calculation ---
            if len(timestamps) > 1:
                tvals = np.array(list(timestamps.values()))
                drift_ms = (tvals.max() - tvals.min()) * 1000.0
            else:
                drift_ms = 0.0

            # --- Draw every camera independently ---
            for name, w in workers.items():
                frame, ts = w.read_latest()
                if frame is None:
                    continue
                # AprilTag overlay
                corners, ids, _ = w.detector.detectMarkers(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
                if ids is not None and len(ids) > 0:
                    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

                # resolution text
                h, wid = frame.shape[:2]
                cv2.putText(frame, f"{name} {wid}x{h}", (10, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"FPS: {fps[name]:.0f}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0), 2)

                cv2.putText(frame, f"Drift: {drift_ms:.1f} ms", (10, 75),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

                cv2.imshow(f"Tuned - {name}", frame)

            # Exit on ESC
            if cv2.waitKey(1) & 0xFF == 27:
                break

            time.sleep(0.001)

    except KeyboardInterrupt:
        pass

    finally:
        for w in workers.values():
            w.stop()
        cv2.destroyAllWindows()



=== Multi-Camera Live View Setup ===
Select cameras to open (comma separated):
1. Kreo Webcam #1
2. Kreo Webcam #2
3. Mobile IP Webcam
Example: 1,2 or 1,3 or 1,2,3
[INFO] Warmup for 1.2 seconds to let cameras settle...
[kreo1] Starting adaptive tuning (~5 minutes)...
[kreo1] Coarse grid: 81 combos
[kreo1] [Coarse 81/81] e=300, f=360, g=18, b=24     
[kreo1] Top coarse candidates:
    (160, 240, 0, 0) score=218.5 {'avg_tags': 2.2083333333333335, 'std_tags': 0.705878097754059, 'fps': 23.667711137355457, 'samples': 24}
    (300, 240, 18, 16) score=198.6 {'avg_tags': 1.9583333333333333, 'std_tags': 0.19982631347136331, 'fps': 23.901857901387764, 'samples': 24}
    (160, 240, 0, 16) score=198.3 {'avg_tags': 2.0, 'std_tags': 0.6454972243679028, 'fps': 23.66644801898144, 'samples': 24}
    (300, 240, 0, 0) score=198.2 {'avg_tags': 1.9565217391304348, 'std_tags': 0.20393111999232305, 'fps': 22.931216802931385, 'samples': 23}
[kreo1] Refining around (160, 240, 0, 0) → 36 combos
[kreo1] [Refine

In [8]:
import cv2, time, json, os, threading, queue, numpy as np, sys
import zmq, msgpack, msgpack_numpy as m
m.patch()
from collections import defaultdict, deque

# =============== CONFIG ===============
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11

CAM_SOURCES = {
    "Mobile":"http://192.168.137.110:8080/video",
    "Kreo1": 0,
    "Kreo2": 4
}

def get_camera_selection():
    print("\n=== Multi-Camera Live View Setup ===")
    print("Select cameras to open (comma separated):")
    print("1. Kreo Webcam #1")
    print("2. Kreo Webcam #2")
    print("3. Mobile IP Webcam")
    print("Example: 1,2 or 1,3 or 1,2,3")
    user_in = input("Cameras to open: ").strip()
    choices = [x.strip() for x in user_in.split(",") if x.strip()]
    selected = []
    for c in choices:
        if c == "1":
            selected.append(("kreo1", CAM_SOURCES["Kreo1"], 0.091))
        elif c == "2":
            selected.append(("kreo2", CAM_SOURCES["Kreo2"], 0.091))
        elif c == "3":
            selected.append(("mobile", CAM_SOURCES["Mobile"], 0.091))
        else:
            print(f"[WARN] Ignoring invalid entry: {c}")
    if not selected:
        print("[ERROR] No valid cameras selected. Exiting.")
        sys.exit(1)
    return selected

TAG4_MOBILE_ID = 4
TAG4_HEIGHT_M = 0.075  # meters
CAPTURE_TARGET_FPS = 60.0
PROCESSING_TARGET_FPS = 40.0
FRAME_QUEUE_MAX = 8
PROCESS_QUEUE_MAX = 8
SHOW_DEBUG_WINDOW = False
DEBUG_PRINT_DETECTIONS = False
APPLY_CAMERA_SETTINGS = True
CALIB_DIR = '../calibration/'  # folder holding camera_calibration_{cam_name}.npz and best_camera_settings_{cam_name}.json
SETTINGS_DIR = './camera_tune_results/'

def load_camera_calib(cam_name):
    path = os.path.join(CALIB_DIR, f'camera_calibration_{cam_name}.npz')
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    calib = np.load(path)
    camera_matrix = calib["cameraMatrix"]
    dist_coeffs = calib['distCoeffs']
    print("[INFO] Loaded calibrated camera parameters")
    return camera_matrix, dist_coeffs

CAMPROP_MAP = {
    'frame_width': cv2.CAP_PROP_FRAME_WIDTH,
    'frame_height': cv2.CAP_PROP_FRAME_HEIGHT,
    'fps': cv2.CAP_PROP_FPS,
    'exposure': cv2.CAP_PROP_EXPOSURE,
    'gain': cv2.CAP_PROP_GAIN,
    'focus': cv2.CAP_PROP_FOCUS,
    'brightness': cv2.CAP_PROP_BRIGHTNESS,
    'contrast': cv2.CAP_PROP_CONTRAST,
    'saturation': cv2.CAP_PROP_SATURATION,
}

def apply_settings_to_capture(cap, settings):
    for k, v in settings.items():
        if k in CAMPROP_MAP:
            try:
                cap.set(CAMPROP_MAP[k], float(v))
            except Exception:
                pass

def create_detector():
    """Setup AprilTag detector with tuned parameters."""
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 7
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 6.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

def estimate_pose_apriltag(corners, tag_size, cam_mtx, cam_dist):
    half = tag_size / 2.0
    objp = np.array([
        [-half,  half, 0.0],
        [ half,  half, 0.0],
        [ half, -half, 0.0],
        [-half, -half, 0.0]
    ], dtype=np.float32)

    imgp = corners.reshape(4,2).astype(np.float32)

    ok, rvec, tvec = cv2.solvePnP(
        objp, imgp, cam_mtx, cam_dist,
        flags=cv2.SOLVEPNP_ITERATIVE
    )

    if not ok:
        raise RuntimeError("solvePnP failed")

    R, _ = cv2.Rodrigues(rvec)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = tvec.reshape(3)

    #debug
    # reprojection
    # imgpts, _ = cv2.projectPoints(objp, rvec, tvec, cam_mtx, cam_dist)
    # imgpts = imgpts.reshape(-1,2)

    # # compute reprojection error against detected corners 'img_corners' (4x2)
    # reproj_err = np.linalg.norm(imgpts - corners, axis=1).mean()

    # # compute camera-frame coords of each object corner
    # # X_cam = R @ X_tag + t
    # cam_pts = (R @ objp.T).T + tvec.reshape(1,3)

    # print("=== REPROJ DEBUG ===")
    # print("rvec:", rvec.ravel())
    # print("tvec:", tvec.ravel())
    # print("R det:", np.linalg.det(R))
    # print("reproj_err px:", reproj_err)
    # print("camera-frame corners (x,y,z):")
    # for i,p in enumerate(cam_pts):
    #     print(f"  corner {i}: {p}")
    return T
    

class CameraWorker:
    def __init__(self, name, device, tag_size, display_manager=None):
        self.name = name
        self.device = device
        self.tag_size = float(tag_size)
        self.display_manager = display_manager
        self.cap = None
        self.running = threading.Event()
        self.frame_queue = queue.Queue(maxsize=FRAME_QUEUE_MAX)
        self.process_queue = queue.Queue(maxsize=PROCESS_QUEUE_MAX)
        self.capture_thread = None
        self.process_thread = None
        self.detector = create_detector()
        self.cam_mtx = None
        self.cam_dist = None
        self.settings = {}
        self.stats = {'captured': 0, 'processed': 0, 'last_proc_time': 0.0}

    def open_capture(self):
        if isinstance(self.device, int):
            cap = cv2.VideoCapture(self.device, cv2.CAP_V4L2)
        else:
            cap = cv2.VideoCapture(int(self.device),cv2.CAP_V4L2)
        
        if not cap.isOpened():
            cap = cv2.VideoCapture(self.device)
        
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
        cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_FPS, CAPTURE_TARGET_FPS)
        return cap
    
    def load_calib_and_settings(self):
        try:
            self.cam_mtx, self.cam_dist = load_camera_calib(self.name)
        except Exception as e:
            print(f'[{self.name}] camera calibration load failed: {e}')
            raise
        sfile = os.path.join(SETTINGS_DIR, f'best_camera_settings_{self.name}.json')
        if os.path.exists(sfile):
            try:
                with open(sfile, 'r') as f:
                    self.settings = json.load(f)
            except Exception:
                self.settings = {}
        else:
            self.settings = {}

    def start(self):
        self.running.set()
        self.load_calib_and_settings()
        self.cap = self.open_capture()
        if APPLY_CAMERA_SETTINGS and self.settings:
            apply_settings_to_capture(self.cap, self.settings)
        self.capture_thread = threading.Thread(target=self._capture_loop, name=f'cap-{self.name}', daemon=True)
        self.process_thread = threading.Thread(target=self._process_loop, name=f'proc-{self.name}', daemon=True)
        self.capture_thread.start()
        self.process_thread.start()
    
    def stop(self):
        self.running.clear()
        if self.capture_thread:
            self.capture_thread.join(timeout=1.0)
        if self.process_thread:
            self.process_thread.join(timeout=1.0)
        if self.cap:
            self.cap.release()

    def _capture_loop(self):
        target_dt = 1.0 / CAPTURE_TARGET_FPS
        while self.running.is_set():
            t0 = time.time()
            ret, frame = self.cap.read()
            t1 = time.time()
            print(f"[{self.name}] cap read time: {t1-t0:.3f}s")
            if not ret:
                time.sleep(0.005)
                continue
            try:
                self.frame_queue.put_nowait((time.time(), frame))
                self.stats['captured'] += 1
            except queue.Full:
                try:
                    self.frame_queue.get_nowait()
                    self.frame_queue.put_nowait((time.time(), frame))
                except Exception:
                    pass
            dt = time.time() - t0
            sl = max(0.0, target_dt - dt)
            if sl > 0:
                time.sleep(sl)

    def _process_loop(self):
        target_dt = 1.0 / PROCESSING_TARGET_FPS
        while self.running.is_set():
            try:
                ts, frame = self.frame_queue.get(timeout=0.5)
            except queue.Empty:
                continue
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, _ = self.detector.detectMarkers(gray)
            results = []
            overlays = []   # <=== NEW for DisplayManager

            if ids is not None:
                for i, tid in enumerate(ids.flatten()):
                    c = corners[i][0]  # 4x2

                    if DEBUG_PRINT_DETECTIONS:
                        print(f"[{self.name}] tag {tid}, corners: {c}")

                    try:
                        t_before = time.time()
                        pose = estimate_pose_apriltag(c, self.tag_size, self.cam_mtx, self.cam_dist)
                        t_after = time.time()
                        print("pose time:", t_after - t_before)
                        results.append({'id': int(tid), 'pose_cam_tag': pose, 'timestamp': ts})
                    except:
                        continue
                    overlays.append({'id': int(tid), 'corners': c})
            # Submit to display manager
            if SHOW_DEBUG_WINDOW and self.display_manager:
                try:
                    self.display_manager.submit({
                        self.name: (ts, frame.copy(), overlays)
                    })
                except:
                    pass
            try:
                self.process_queue.put_nowait((ts, frame, results))
            except queue.Full:
                _ = self.process_queue.get_nowait()
                self.process_queue.put_nowait((ts, frame, results))

            self.stats['processed'] += len(results)

            time.sleep(max(0, target_dt - (time.time() - ts)))

# central mapper
class WorldMapper:
    def __init__(self):
        self.lock = threading.RLock()
        self.tag_world_poses = {} # tag_id -> 4x4 pose in world (tag -> world)
        self.camera_world_poses = {}  # cam_name -> 4x4 pose in world (camera -> world)
        self.history = defaultdict(deque)

    @staticmethod
    def invert_pose(T):
        R = T[:3, :3]
        t = T[:3, 3]
        Tinv = np.eye(4)
        Tinv[:3, :3] = R.T
        Tinv[:3, 3] = -R.T.dot(t)
        return Tinv
    
    def _recenter_on_tag1(self):
        """
        If tag 1 exists, transform all stored poses so that tag1 becomes the world origin.
        That is, compute S = inv(T_world_tag1) and replace every pose by S.dot(pose).
        After this, T_world_tag1 will be identity.
        """
        TAG1 = 1
        if TAG1 not in self.tag_world_poses:
            return

        T_world_tag1 = self.tag_world_poses[TAG1]
        S = self.invert_pose(T_world_tag1)  # S = inv(T_world_tag1)

        # apply S to all tags and cameras
        for tid in list(self.tag_world_poses.keys()):
            self.tag_world_poses[tid] = S.dot(self.tag_world_poses[tid])

        for cname in list(self.camera_world_poses.keys()):
            self.camera_world_poses[cname] = S.dot(self.camera_world_poses[cname])

    def add_observation(self, cam_name, cam_T_tag, tag_id, timestamp):
        with self.lock:
            cam_T_tag = np.array(cam_T_tag, dtype=float)
            tag_T_cam = self.invert_pose(cam_T_tag)  # tag_T_cam maps camera -> tag

            # Case A: we already know this tag's world pose -> compute camera world
            if tag_id in self.tag_world_poses:
                T_world_tag = self.tag_world_poses[tag_id]
                # T_world_cam = T_world_tag . T_tag_cam
                T_world_cam = T_world_tag.dot(tag_T_cam)
                self.camera_world_poses[cam_name] = T_world_cam

            # Case B: we know this camera's world pose -> compute tag world
            elif cam_name in self.camera_world_poses:
                T_world_cam = self.camera_world_poses[cam_name]
                # T_world_tag = T_world_cam . T_cam_tag
                T_world_tag = T_world_cam.dot(cam_T_tag)

                # If this is TAG4 (mobile), force its height to TAG4_HEIGHT_M (z)
                if tag_id == TAG4_MOBILE_ID:
                    T_world_tag = np.array(T_world_tag, dtype=float)
                    T_world_tag[2, 3] = TAG4_HEIGHT_M

                self.tag_world_poses[tag_id] = T_world_tag

            # Case C: neither tag nor camera is known yet
            else:
                # If this is tag 1, make it the world origin
                if tag_id == 1:
                    T_world_tag = np.eye(4)
                    # camera world = T_world_tag . T_tag_cam = I . T_tag_cam = T_tag_cam
                    T_world_cam = T_world_tag.dot(tag_T_cam)
                    self.tag_world_poses[tag_id] = T_world_tag
                    self.camera_world_poses[cam_name] = T_world_cam

                # If this is TAG4 (mobile) and first observation, place tag relative to camera
                # then set its z to TAG4_HEIGHT_M
                elif tag_id == TAG4_MOBILE_ID:
                    # choose camera as temporary world (i.e., set T_world_cam = I)
                    # So tag_world = T_world_cam . T_cam_tag = I . T_cam_tag = cam_T_tag
                    T_world_tag = cam_T_tag.copy()
                    T_world_tag = np.array(T_world_tag, dtype=float)
                    T_world_tag[2, 3] = TAG4_HEIGHT_M
                    self.tag_world_poses[tag_id] = T_world_tag
                    # camera world is then T_world_cam = T_world_tag . T_tag_cam
                    T_world_cam = T_world_tag.dot(tag_T_cam)
                    self.camera_world_poses[cam_name] = T_world_cam

                else:
                    # Default: choose camera as temporary world origin.
                    # So T_world_cam = Identity, and T_world_tag = I . cam_T_tag = cam_T_tag
                    T_world_cam = np.eye(4)
                    T_world_tag = cam_T_tag.copy()
                    # store both
                    self.camera_world_poses[cam_name] = T_world_cam
                    self.tag_world_poses[tag_id] = T_world_tag

            # record history
            self.history[tag_id].append((timestamp, cam_name))
            # keep limited history
            if len(self.history[tag_id]) > 200:
                self.history[tag_id].popleft()

            # IMPORTANT: if tag 1 appears at any time, recenter the whole map so tag1 is origin
            if 1 in self.tag_world_poses:
                # Recenter only if tag1 is not already identity
                T_world_tag1 = self.tag_world_poses[1]
                if not np.allclose(T_world_tag1, np.eye(4), atol=1e-6):
                    self._recenter_on_tag1()

    def get_map_snapshot(self):
        with self.lock:
            tags = {tid: T.copy() for tid, T in self.tag_world_poses.items()}
            cams = {c: T.copy() for c, T in self.camera_world_poses.items()}
        return {'tags': tags, 'cameras': cams}

class DisplayManager:
    def __init__(self, target_fps=25, downscale_w=640):
        self.q = queue.Queue(maxsize=1)
        self.running = threading.Event()
        self.thread = None
        self.target_dt = 1.0 / target_fps
        self.downscale_w = downscale_w

    def start(self):
        self.running.set()
        self.thread = threading.Thread(target=self._loop, daemon=True)
        self.thread.start()

    def stop(self):
        self.running.clear()
        if self.thread:
            self.thread.join(timeout=1.0)
        try:
            cv2.destroyAllWindows()
        except:
            pass

    def submit(self, frames_dict):
        """frames_dict = {cam_name: (ts, full_frame, overlays)}"""
        try:
            self.q.put_nowait(frames_dict)
        except queue.Full:
            try:
                _ = self.q.get_nowait()
                self.q.put_nowait(frames_dict)
            except:
                pass

    def _annotate_frame(self, frame, overlays):
        for o in overlays:
            c = o['corners']
            tid = o['id']
            pts = c.astype(int)
            cv2.polylines(frame, [pts], True, (0,255,0), 2)
            cv2.putText(frame, f"id:{tid}", tuple(pts[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        return frame

    def _loop(self):
        while self.running.is_set():
            try:
                frames_dict = self.q.get(timeout=0.1)
            except queue.Empty:
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    self.running.clear()
                continue

            tiles = []

            for cam_name, (ts, full_frame, overlays) in frames_dict.items():
                h, w = full_frame.shape[:2]
                scale = min(1.0, self.downscale_w / float(w))
                if scale < 1.0:
                    small = cv2.resize(full_frame, (int(w*scale), int(h*scale)), cv2.INTER_LINEAR)
                    scaled_overlays = []
                    for o in overlays:
                        scaled_overlays.append({
                            'id': o['id'],
                            'corners': o['corners'] * scale
                        })
                else:
                    small = full_frame.copy()
                    scaled_overlays = overlays

                small = self._annotate_frame(small, scaled_overlays)
                cv2.putText(small, cam_name, (10,25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
                tiles.append(small)

            if len(tiles) == 1:
                out = tiles[0]
            else:
                maxh = max(t.shape[0] for t in tiles)
                padded = []
                for t in tiles:
                    if t.shape[0] < maxh:
                        pad = np.zeros((maxh - t.shape[0], t.shape[1], 3), np.uint8)
                        t = np.vstack([t, pad])
                    padded.append(t)
                out = np.hstack(padded)

            cv2.imshow("MultiCam Debug", out)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.running.clear()

            time.sleep(self.target_dt)


class SnapshotPublisher:
    def __init__(self, port=5557):
        ctx = zmq.Context()
        self.sock = ctx.socket(zmq.PUB)
        self.sock.bind(f"tcp://*:{port}")
    
    def send(self, snapshot):
        # snapshot = {'tags': {tid: np.array(4x4)}, 'cameras': {name: np.array(4x4)}}
        # convert to pure dict of lists
        msg = {
            "tags": {tid: snapshot['tags'][tid] for tid in snapshot['tags']},
            "cameras": {c: snapshot['cameras'][c] for c in snapshot['cameras']}
        }
        packed = msgpack.packb(msg, default=m.encode)
        self.sock.send(packed)

    

In [9]:
def main():
    display = DisplayManager(target_fps=25, downscale_w=640)
    if SHOW_DEBUG_WINDOW:
        display.start()

    workers = []
    for name, dev, tag_size in get_camera_selection():
        w = CameraWorker(name, dev, tag_size, display_manager=display)
        try:
            w.start()
        except Exception as e:
            print(f'Failed to start camera {name}: {e}')
            continue
        workers.append(w)
    
    mapper = WorldMapper()
    pub = SnapshotPublisher(port=5557)
    map_thread_stop = threading.Event()

    def mapper_loop():
        while not map_thread_stop.is_set():
            for w in workers:
                try:
                    ts, frame, dets = w.process_queue.get_nowait()
                except queue.Empty:
                    continue

                for d in dets:
                    mapper.add_observation(w.name, d['pose_cam_tag'], d['id'], d['timestamp'])
                    snapshot = mapper.get_map_snapshot()
                    pub.send(snapshot)
            time.sleep(0.002)
    
    mt = threading.Thread(target=mapper_loop, name='mapper', daemon=True)
    mt.start()

    try:
        last_print = time.time()
        while True:
            now = time.time()
            if now - last_print > 1.0:
                last_print = now
                snap = mapper.get_map_snapshot()
                print('--- status ---')
                for w in workers:
                    print(f'{w.name}: cap {w.stats["captured"]} proc {w.stats["processed"]}')
                print('tags known:', list(snap['tags'].keys()))
            # time.sleep(0.01)
    except KeyboardInterrupt:
        pass
    finally:
        map_thread_stop.set()
        for w in workers:
            w.stop()
        if SHOW_DEBUG_WINDOW:
            cv2.destroyAllWindows()
        print('exiting main')

if __name__ == '__main__':
    main()


=== Multi-Camera Live View Setup ===
Select cameras to open (comma separated):
1. Kreo Webcam #1
2. Kreo Webcam #2
3. Mobile IP Webcam
Example: 1,2 or 1,3 or 1,2,3
[INFO] Loaded calibrated camera parameters
[kreo1] cap read time: 0.174s
[kreo1] cap read time: 0.037s
pose time: 0.01063680648803711
[kreo1] cap read time: 0.026s
[kreo1] cap read time: 0.012s
pose time: 0.005784511566162109
[kreo1] cap read time: 0.020s
pose time: 0.017647743225097656
[kreo1] cap read time: 0.029s
pose time: 0.005591869354248047
[kreo1] cap read time: 0.037s
pose time: 0.027540922164916992
pose time: 0.0059108734130859375
[kreo1] cap read time: 0.045s
[kreo1] cap read time: 0.023s
[kreo1] cap read time: 0.023s
pose time: 0.005336761474609375
[kreo1] cap read time: 0.016s
[kreo1] cap read time: 0.015s
pose time: 0.005643129348754883
[kreo1] cap read time: 0.021s
[kreo1] cap read time: 0.030s
pose time: 0.005594730377197266
[kreo1] cap read time: 0.021s
[kreo1] cap read time: 0.019s
pose time: 0.00550532341